<a href="https://colab.research.google.com/github/nalrob/Procesamiento_de_datos/blob/main/TMDB_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python -m pip install requests

**API extraction**


In [16]:
import json
import requests
from pprint import pprint

api_key = '{{c15924a73434d6c36c7b2afaeb15f53a}}'
language = 'en-US'

response = requests.get("https://api.themoviedb.org/3/search/movie?query=Whiplash&language=de-DE&region=DE&api_key=c15924a73434d6c36c7b2afaeb15f53a",
                        params={'api_key':api_key,
                                'language':language})


pprint(response.json())

{'page': 1,
 'results': [{'adult': False,
              'backdrop_path': '/vNXGrknx4GjWLgmuNTftWZluIUl.jpg',
              'genre_ids': [18, 10402],
              'id': 244786,
              'original_language': 'en',
              'original_title': 'Whiplash',
              'overview': 'Under the direction of a ruthless instructor, a '
                          'talented young drummer begins to pursue perfection '
                          'at any cost, even his humanity.',
              'popularity': 68.419,
              'poster_path': '/7fn624j5lj3xTme2SgiLCeuedmO.jpg',
              'release_date': '2015-02-19',
              'title': 'Whiplash',
              'video': False,
              'vote_average': 8.383,
              'vote_count': 14173},
             {'adult': False,
              'backdrop_path': '/sO70LyX3CHDNimbUwdGqL6pjrD3.jpg',
              'genre_ids': [18, 10402],
              'id': 367412,
              'original_language': 'en',
              'original_title':

**Updaload the data and create the dataset**

Download it into a CSV format

In [10]:
import requests
import csv

api_key = 'c15924a73434d6c36c7b2afaeb15f53a'
language = 'en-US'
pages_to_extract = 150

response = requests.get("https://api.themoviedb.org/3/discover/movie",
                        params={'api_key': api_key,
                                'language': language,
                                'page': 1})

if response.status_code == 200:
    data = response.json()

    # Verificar si la clave 'total_pages' existe en el JSON
    total_pages = data.get('total_pages', 0)

    if total_pages > 0:

        pages_to_extract = min(pages_to_extract, total_pages)

        all_movies = []

        for page in range(1, pages_to_extract + 1):

            response = requests.get("https://api.themoviedb.org/3/discover/movie",
                                    params={'api_key': api_key,
                                            'language': language,
                                            'page': page})

            if response.status_code == 200:
                data = response.json()


                if 'results' in data:
                    movies = data['results']


                    all_movies.extend(movies)
                else:
                    print(f"La clave 'results' no está presente en la respuesta JSON de la página {page}.")
            else:
                print(f"Error en la solicitud de la página {page}. Código de estado: {response.status_code}")


        csv_file = 'movies_dataset.csv'

        with open(csv_file, 'w', newline='', encoding='utf-8') as file:

            csv_writer = csv.writer(file)
            csv_writer.writerow(['ID', 'Title', 'Release Date', 'Overview', 'Vote Average', 'Genre IDs', 'Original Language','VoteCount', 'Popularity', 'Adult'])


            for movie in all_movies:
                id = movie.get('id', '')
                title = movie.get('title', '')
                release_date = movie.get('release_date', '')
                overview = movie.get('overview', '')
                vote_average = movie.get('vote_average', '')
                genre_ids = movie.get('genre_ids', '')
                original_language = movie.get('original_language', '')
                vote_count = movie.get('vote_count', '')
                popularity = movie.get('popularity', '')
                adult = movie.get('adult', '')
                csv_writer.writerow([id, title, release_date, overview, vote_average, genre_ids, original_language,vote_count, popularity, adult])

        print(f"Dataset creado y guardado en {csv_file}")
    else:
        print("Mistake")
else:
    print(f"Error: {response.status_code}")



Dataset creado y guardado en movies_dataset.csv


**Updaload the data and create the dataset**

In [1]:
import pandas as pd
import requests

api_key = 'c15924a73434d6c36c7b2afaeb15f53a'
language = 'en-US'
pages_to_extract = 100

genres_url = f"https://api.themoviedb.org/3/genre/movie/list"
genres_params = {'api_key': api_key, 'language': language}
genres_response = requests.get(genres_url, params=genres_params)

if genres_response.status_code == 200:
    genres_data = genres_response.json()
    genres_dict = {genre['id']: genre['name'] for genre in genres_data['genres']}
else:
    print(f"Error en la solicitud de la lista de géneros. Código de estado: {genres_response.status_code}")
    exit()

response = requests.get("https://api.themoviedb.org/3/discover/movie",
                        params={'api_key': api_key,
                                'language': language,
                                'page': 1})

if response.status_code == 200:

    data = response.json()
    total_pages = data.get('total_pages', 0)

    if total_pages > 0:

        pages_to_extract = min(pages_to_extract, total_pages)
        all_movies = []

        for page in range(1, pages_to_extract + 1):
            response = requests.get("https://api.themoviedb.org/3/discover/movie",
                                    params={'api_key': api_key,
                                            'language': language,
                                            'page': page})

            if response.status_code == 200:
                data = response.json()

                if 'results' in data:
                    movies = data['results']

                    for movie in movies:
                        genre_ids = movie.get('genre_ids', [])
                        movie['genres'] = [genres_dict.get(genre_id, 'Unknown') for genre_id in genre_ids]

                        movie_id = movie.get('id')
                        budget_url = f"https://api.themoviedb.org/3/movie/{movie_id}"
                        budget_params = {'api_key': api_key, 'language': language}
                        budget_response = requests.get(budget_url, params=budget_params)

                        if budget_response.status_code == 200:
                            budget_data = budget_response.json()
                            movie['budget'] = budget_data.get('budget')
                        else:
                            print(f"Error en la solicitud de presupuesto para la película {movie_id}. Código de estado: {budget_response.status_code}")

                    all_movies.extend(movies)
                else:
                    print(f"La clave 'results' no está presente en la respuesta JSON de la página {page}.")
            else:
                print(f"Error en la solicitud de la página {page}. Código de estado: {response.status_code}")

        df = pd.DataFrame(all_movies)

        print(df)

    else:
        print("Mistake")
else:
    print(f"Error: {response.status_code}")


      adult                     backdrop_path                 genre_ids  \
0     False  /r9oTasGQofvkQY5vlUXglneF64Z.jpg                  [28, 35]   
1     False  /5a4JdoFwll5DRtKMe7JLuGQ9yJm.jpg             [18, 878, 28]   
2     False  /sRLC052ieEzkQs9dEtPMfFxYkej.jpg                     [878]   
3     False  /gg4zZoTggZmpAQ32qIrP5dtnkEZ.jpg                  [28, 80]   
4     False  /jXJxMcVoEuXzym3vFnjqDW4ifo6.jpg              [28, 12, 14]   
...     ...                               ...                       ...   
1995  False  /rrDZljxZ0DSWVn6M5fkJwEOVOWn.jpg          [14, 16, 35, 80]   
1996  False  /eCgd6Q7zeFwpLrAm44HVv8WaOb1.jpg  [28, 878, 12, 35, 10751]   
1997  False  /mM4WeXqJd2tu1gCF5GYyAZf2Npl.jpg              [12, 28, 53]   
1998  False  /hXTWVJMsI9BkxMLliqL1j0FT55t.jpg          [28, 12, 80, 53]   
1999  False  /uFwnUQ3ZwE5EbBEDBzs1bRPlOx8.jpg             [28, 12, 878]   

           id original_language  \
0     1029575                en   
1      695721                

In [2]:
df.columns

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count',
       'genres', 'budget'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              2000 non-null   bool   
 1   backdrop_path      1981 non-null   object 
 2   genre_ids          2000 non-null   object 
 3   id                 2000 non-null   int64  
 4   original_language  2000 non-null   object 
 5   original_title     2000 non-null   object 
 6   overview           2000 non-null   object 
 7   popularity         2000 non-null   float64
 8   poster_path        2000 non-null   object 
 9   release_date       1998 non-null   object 
 10  title              2000 non-null   object 
 11  video              2000 non-null   bool   
 12  vote_average       2000 non-null   float64
 13  vote_count         2000 non-null   int64  
 14  genres             2000 non-null   object 
 15  budget             2000 non-null   int64  
dtypes: bool(2), float64(2), 

In [6]:
df['original_language'].value_counts()

en    1608
ja     114
es      71
ko      45
fr      36
it      19
zh      16
ru      12
pl       9
de       8
pt       7
tl       7
no       7
hi       6
cn       4
sv       4
th       3
ar       3
te       2
id       2
fi       2
tr       2
nl       2
da       2
ca       2
uk       2
ml       1
yo       1
is       1
mn       1
ig       1
Name: original_language, dtype: int64

***Transformation & Cleaning***

In [7]:
movies=df.drop(columns=['adult', 'backdrop_path', 'original_language',
       'original_title', 'popularity', 'poster_path',
       'release_date', 'video', 'vote_average','genre_ids', 'vote_count',
       'budget'])

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2000 non-null   int64 
 1   overview  2000 non-null   object
 2   title     2000 non-null   object
 3   genres    2000 non-null   object
dtypes: int64(1), object(3)
memory usage: 62.6+ KB


In [8]:
movies.isnull().sum()

id          0
overview    0
title       0
genres      0
dtype: int64

In [9]:
movies.dropna(inplace=True)
movies.isnull().sum()

id          0
overview    0
title       0
genres      0
dtype: int64

 Program to convert String to a List

In [10]:
for i in movies['genres'][0]:
   for k,v in movies['genres'].items():
     print(v)

['Action', 'Comedy']
['Drama', 'Science Fiction', 'Action']
['Science Fiction']
['Action', 'Crime']
['Action', 'Adventure', 'Fantasy']
['Horror', 'Mystery', 'Thriller']
['Horror']
['Horror', 'Mystery']
['Thriller', 'Action']
['Action', 'Comedy']
['Action', 'Adventure', 'Thriller']
['Action', 'Adventure', 'Science Fiction']
['Drama', 'Romance']
['Horror', 'Mystery']
['Drama', 'History']
['Action', 'War']
['Comedy', 'Family', 'Fantasy']
['Horror']
['Animation', 'Action', 'Fantasy']
['Action', 'Comedy', 'Crime']
['Animation', 'Comedy', 'Family']
['Animation', 'Family', 'Music', 'Fantasy', 'Comedy']
['Thriller', 'Drama']
['Action', 'Crime', 'Drama', 'Thriller']
['Drama', 'History']
['Action', 'Adventure', 'Science Fiction']
['Action', 'Thriller', 'Crime']
['Horror', 'Thriller']
['Action', 'Crime', 'Thriller']
['Animation', 'Family', 'Adventure', 'Fantasy', 'Comedy']
['Animation', 'Adventure', 'Comedy', 'Family']
['Action', 'Crime', 'Drama']
['Horror']
['Thriller', 'Science Fiction', 'Actio

In [11]:
type(movies)

pandas.core.frame.DataFrame

In [12]:
import ast
def convert(obj):
 L=[]
 for i in ast.literal_eval(obj):
  L.append(i['name'])
 return L

In [13]:
movies['overview']=movies['overview'].tolist()
type(movies['overview'])

pandas.core.series.Series

In [14]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [15]:
movies.head()

,id,overview,title,genres
0,1029575,"[Dan, Morgan, is, many, things:, a, devoted, h...",The Family Plan,"[Action, Comedy]"
1,695721,"[64, years, before, he, becomes, the, tyrannic...",The Hunger Games: The Ballad of Songbirds & Sn...,"[Drama, Science Fiction, Action]"
2,848326,"[When, a, peaceful, colony, on, the, edge, of,...",Rebel Moon - Part One: A Child of Fire,[Science Fiction]
3,891699,"[A, tormented, father, witnesses, his, young, ...",Silent Night,"[Action, Crime]"
4,572802,"[Black, Manta,, still, driven, by, the, need, ...",Aquaman and the Lost Kingdom,"[Action, Adventure, Fantasy]"


In [16]:
movies['tags']=movies['overview']+movies['genres']
movies.head()

,id,overview,title,genres,tags
0,1029575,"[Dan, Morgan, is, many, things:, a, devoted, h...",The Family Plan,"[Action, Comedy]","[Dan, Morgan, is, many, things:, a, devoted, h..."
1,695721,"[64, years, before, he, becomes, the, tyrannic...",The Hunger Games: The Ballad of Songbirds & Sn...,"[Drama, Science Fiction, Action]","[64, years, before, he, becomes, the, tyrannic..."
2,848326,"[When, a, peaceful, colony, on, the, edge, of,...",Rebel Moon - Part One: A Child of Fire,[Science Fiction],"[When, a, peaceful, colony, on, the, edge, of,..."
3,891699,"[A, tormented, father, witnesses, his, young, ...",Silent Night,"[Action, Crime]","[A, tormented, father, witnesses, his, young, ..."
4,572802,"[Black, Manta,, still, driven, by, the, need, ...",Aquaman and the Lost Kingdom,"[Action, Adventure, Fantasy]","[Black, Manta,, still, driven, by, the, need, ..."


In [ ]:
movies=movies.drop(columns=['overview','genres'])

In [19]:
new_df=movies
new_df

,id,title,tags
0,1029575,The Family Plan,"[Dan, Morgan, is, many, things:, a, devoted, h..."
1,695721,The Hunger Games: The Ballad of Songbirds & Sn...,"[64, years, before, he, becomes, the, tyrannic..."
2,848326,Rebel Moon - Part One: A Child of Fire,"[When, a, peaceful, colony, on, the, edge, of,..."
3,891699,Silent Night,"[A, tormented, father, witnesses, his, young, ..."
4,572802,Aquaman and the Lost Kingdom,"[Black, Manta,, still, driven, by, the, need, ..."
...,...,...,...
1995,856,Who Framed Roger Rabbit,"['Toon, star, Roger, is, worried, that, his, w..."
1996,88751,Journey to the Center of the Earth,"[On, a, quest, to, find, out, what, happened, ..."
1997,658,Goldfinger,"[Special, agent, 007, comes, face, to, face, w..."
1998,606402,Yaksha: Ruthless Operations,"[Nicknamed, after, a, human-devouring, spirit,..."


In [20]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
new_df.head()

,id,title,tags
0,1029575,The Family Plan,"Dan Morgan is many things: a devoted husband, ..."
1,695721,The Hunger Games: The Ballad of Songbirds & Sn...,64 years before he becomes the tyrannical pres...
2,848326,Rebel Moon - Part One: A Child of Fire,When a peaceful colony on the edge of the gala...
3,891699,Silent Night,A tormented father witnesses his young son die...
4,572802,Aquaman and the Lost Kingdom,"Black Manta, still driven by the need to aveng..."


**Exploratory Analysis**

In [23]:
from IPython.display import HTML

In [24]:
popular_movies = df.nlargest(n=5, columns=['popularity'])[['id', 'title']]
getList_name = {}
for x, xRows in popular_movies.iterrows():
    #print(xRows['id'])
    getResponse = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8'.format(xRows['id']) )
    getData = getResponse.json()
    getPath = "http://image.tmdb.org/t/p/w500" + getData['poster_path']
    getList_name[xRows['title']] = getPath

display( HTML(f"""<table>
                    <tr>
                        <td><img src={list(getList_name.values())[0]} style='border-radius:10px; height:600px; width:400px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[1]} style='border-radius:10px; height:600px; width:400px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[2]} style='border-radius:10px; height:600px; width:400px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[3]} style='border-radius:10px; height:600px; width:400px; border:1px solid #999;'></td>
                        <td><img src={list(getList_name.values())[4]} style='border-radius:10px; height:600px; width:400px; border:1px solid #999;'></td>
                    </tr>
                    <tr>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[0]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[1]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[2]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[3]}</div></td>
                        <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{list(getList_name.keys())[4]}</div></td>
                    </tr>
                </table>"""))

,,,,
The Family Plan,The Hunger Games: The Ballad of Songbirds & Snakes,Rebel Moon - Part One: A Child of Fire,Aquaman and the Lost Kingdom,Rewind


In [27]:
def show_poster(sel_movie, mov_name, posterPath):
    display( HTML(f"""<div style="font-size:24px; font-weight:Bold; color:#fff; text-align:center; padding-top:8px; height:12%; width: 100%; border:1px solid #ccc; border-radius:10px; margin-top:10px; background-color:red;">{sel_movie}</div>
                   <table>
                    <tr>
                        <td><img src={posterPath[0]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={posterPath[1]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={posterPath[2]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={posterPath[3]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={posterPath[4]} style='border-radius:10px; width:400px; height:600px;'></td>
                    </tr>
                    <tr>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[0]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[1]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[2]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[3]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{mov_name[4]}</p></td>
                    </tr>
                   </table>""") )

**Preprocessing**

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [29]:
vectors=cv.fit_transform(new_df['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
len(cv.get_feature_names_out())

5000

In [31]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zombies', 'zone', 'zoo'], dtype=object)

Performing stemming in nlp using nltk to stem the words

In [32]:
import nltk

from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [33]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    string = " ".join(y)
    return string


In [35]:
new_df['tags']=new_df['tags'].apply(stem)

In [36]:
len(cv.get_feature_names_out())

5000

In [37]:
len(vectors)

2000

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
prediction=cosine_similarity(vectors)

In [42]:
def recommend(movie):
  movie_index=new_df[new_df['title']==movie].index[0]
  distances=prediction[movie_index]
  movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_df.iloc[i[0]]['title'])

In [43]:
prediction.shape

(2000, 2000)

In [46]:
recommend("The Notebook")

Temptation of Angel
Emancipation
The Longest Ride
Oppenheimer
The Age of Adaline


In [47]:
recommend("Spider-Man 3")

Spider-Man
Venom: Let There Be Carnage
Spider-Man: Homecoming
Spider-Man: Across the Spider-Verse
Spider-Man: Into the Spider-Verse


In [48]:
def recommend_movies_names(xMovie):

    movie_index = new_df[new_df["title"].str.lower() == xMovie.lower()].index[0]
    distances = prediction[movie_index]
    listofMovies = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in listofMovies:
        print(new_df.iloc[i[0]]["title"])

In [50]:
movies_list = {"Hereditary", "Avatar", "Moonfall", "Titanic"}
for x in movies_list:
    print("-"*20, "Recommendations for [", x,"]", "-"*20)
    recommend_movies_names(x)

-------------------- Recommendations for [ Avatar ] --------------------
Mount Adams
Headspace
Guardians of the Galaxy Vol. 3
Godzilla x Kong: The New Empire
Moonfall
-------------------- Recommendations for [ Moonfall ] --------------------
Kraven the Hunter
Mortal Engines
The Matrix Resurrections
Godzilla vs. Kong
Godzilla: The Planet Eater
-------------------- Recommendations for [ Hereditary ] --------------------
Poltergeist
Orphan: First Kill
The Curse of La Llorona
Los Habitantes
La Navidad en sus manos
-------------------- Recommendations for [ Titanic ] --------------------
Temptation of Angel
The Notebook
Looking for Magical Doremi
Your Fault
Revenge Porn: PAIN OF LOVE


In [63]:
movie_name = "The notebook"
names, poster = getRecommended_movies_name(movie_name)

In [64]:
display( HTML(f"""<div style="font-size:24px; font-weight:Bold; color:#fff; text-align:center; padding-top:8px; height:12%; width: 100%; border:1px solid #ccc; border-radius:10px; margin-top:10px; background-color:green;">{movie_name}</div>
                  <table>
                    <tr>
                        <td><img src={poster[0]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={poster[1]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={poster[2]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={poster[3]} style='border-radius:10px; width:400px; height:600px;'></td>
                        <td><img src={poster[4]} style='border-radius:10px; width:400px; height:600px;'></td>
                    </tr>
                    <tr>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[0]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[1]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[2]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[3]}</p></td>
                        <td><p style="text-align:center; font-size:14px; font-weight:bold">{names[4]}</p></td>
                    </tr>
                   </table>""") )

,,,,
Temptation of Angel,Emancipation,The Longest Ride,Oppenheimer,The Age of Adaline


In [53]:
def fetch_movies_poster(movieID):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8'.format(movieID) )
    data = response.json()
    posterPath = "http://image.tmdb.org/t/p/w500" + data['poster_path']
    return posterPath


def getRecommended_movies_name(xMovie):
    movie_index = new_df[new_df["title"].str.lower() == xMovie.lower()].index[0]
    distances = similarity[movie_index]
    listofMovies = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    recomended_movies = []
    movies_poster = []
    for i in listofMovies:
        recomended_movies.append(new_df.iloc[i[0]]["title"])
        movies_poster.append( fetch_movies_poster(new_df.iloc[i[0]]["id"]) )

    return recomended_movies, movies_poster

In [58]:
# Show the Recommended Name and Poster of Movies in List
movies_list = {"Midsommar"}
for x in movies_list:
    names, poster = getRecommended_movies_name(x)
    show_poster(x, names, poster)

,,,,
Jeepers Creepers: Reborn,The Conference,1962 Halloween Massacre,Silver and the Book of Dreams,Lord of Misrule


**Other consults**

In [59]:
max_genres = max(len(genres) for genres in df['genres'])


genre_columns = [f'genres{i}' for i in range(1, max_genres + 1)]
df[genre_columns] = pd.DataFrame(df['genres'].tolist(), index=df.index)

result_df = df[['id'] + genre_columns]
print(result_df)

           id          genres1          genres2          genres3   genres4  \
0     1029575           Action           Comedy             None      None   
1      695721            Drama  Science Fiction           Action      None   
2      848326  Science Fiction             None             None      None   
3      891699           Action            Crime             None      None   
4      572802           Action        Adventure          Fantasy      None   
...       ...              ...              ...              ...       ...   
1995      856          Fantasy        Animation           Comedy     Crime   
1996    88751           Action  Science Fiction        Adventure    Comedy   
1997      658        Adventure           Action         Thriller      None   
1998   606402           Action        Adventure            Crime  Thriller   
1999   200085           Action        Adventure  Science Fiction      None   

     genres5 genres6 genres7  
0       None    None    None  
1

In [60]:
max_genres = max(len(genres) for genres in df['genres'])

genre_columns = [f'genre{i}' for i in range(1, max_genres + 1)]
df[genre_columns] = pd.DataFrame(df['genres'].tolist(), index=df.index)

# Utilizar get_dummies para crear la tabla binaria
binary_table = pd.get_dummies(df[genre_columns].stack()).sum(level=0)

# Concatenar la tabla binaria con la columna 'id'
result_table = pd.concat([df[['id']], binary_table], axis=1)

# Imprimir el resultado final
print(result_table)

           id  Action  Adventure  Animation  Comedy  Crime  Documentary  \
0     1029575     1.0        0.0        0.0     1.0    0.0          0.0   
1      695721     1.0        0.0        0.0     0.0    0.0          0.0   
2      848326     0.0        0.0        0.0     0.0    0.0          0.0   
3      891699     1.0        0.0        0.0     0.0    1.0          0.0   
4      572802     1.0        1.0        0.0     0.0    0.0          0.0   
...       ...     ...        ...        ...     ...    ...          ...   
1995      856     0.0        0.0        1.0     1.0    1.0          0.0   
1996    88751     1.0        1.0        0.0     1.0    0.0          0.0   
1997      658     1.0        1.0        0.0     0.0    0.0          0.0   
1998   606402     1.0        1.0        0.0     0.0    1.0          0.0   
1999   200085     1.0        1.0        0.0     0.0    0.0          0.0   

      Drama  Family  Fantasy  History  Horror  Music  Mystery  Romance  \
0       0.0     0.0      

<ipython-input-60-e31b196cb59c>:7: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  binary_table = pd.get_dummies(df[genre_columns].stack()).sum(level=0)


In [61]:
result_table.to_csv('genres.csv', index=False)

In [28]:
import requests
from IPython.display import HTML, display


movie_name = input("Ingrese el nombre de la película: ")

search_response = requests.get(f'https://api.themoviedb.org/3/search/movie?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8&query={movie_name}')
search_data = search_response.json()


if 'results' in search_data and search_data['results']:
    movie_id = search_data['results'][0]['id']

    get_response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key=c0bda0be71f7815fd6ba2eb5f5c86fd8')
    get_data = get_response.json()


    if 'title' in get_data:

        poster_path = f"http://image.tmdb.org/t/p/w500{get_data['poster_path']}"


        display_data = {
            'poster_path': poster_path,
            'title': get_data['title'],
            'overview': get_data['overview']
        }

        display(HTML(f"""<table>
                            <tr>
                                <td><img src={display_data['poster_path']} style='border-radius:10px; height:600px; width:400px; border:1px solid #999;'></td>
                            </tr>
                            <tr>
                                <td><div style="height:40px; padding-top:15px; text-align:center; font-size:14px; font-weight:bold; border:1px solid #ccc; border-radius:10px;">{display_data['title']}</div></td>
                            </tr>
                            <tr>
                                <td><p style="text-align: center;">{display_data['overview']}</p></td>
                            </tr>
                        </table>"""))
    else:
        print(f"No se encontraron detalles para la película con nombre {movie_name}.")
else:
    print(f"No se encontraron resultados para la búsqueda de {movie_name}.")


Ingrese el nombre de la película: Avatar


""
Avatar
"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization."
